Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock, Germany

Tutorial Digital Signal Processing (Course #24505),
**Parametric Windows: Dolph-Chebyshev, Slepian and Kaiser-Bessel Window**,
Winter Semester 2019/20

Feel free to contact lecturer frank.schultz@uni-rostock.de

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

# Parametric Windows: Dolph-Chebyshev, Slepian, Kaiser-Bessel Window

There are numerous window types, all developed for special requirements. In the *DFT/Windowing* lecture / tutorial we've learned about two very simple and very often used **non-parametric** windows, the Hanning (or more correct termed Hann after the meteorologist 'von Hann') and the Hamming window. Non-parametric means that by desired window length $M$, the window signal and its spectrum are fully determined. In other words, if another spectral characteristics - very often you ask for other spectral resolution - is required, then the only variable to change this is $M$.

The **Hann** window is **not optimal**, since it does not use two of its potential zeros to shape the sidelobes of its DTFT spectrum. The Hamming window introduces two additional zeros to the Hann window to reduce the level of the first sidelobe.
Note that the Hann window is still often used nowadays, not due to its non-optimum spectrum, but rather due to its simple calculation of the window signal $w[k]$ requiring only a cosine and weight of $1/2$.

So called **parametric** windows have additional parameters, that allow to meet certain constraints for a given overall design criterion. Two of the most prominent - in fact with these you probably can manage the majority of all windowing applications - are the **Dolph-Chebyshev** and the **Kaiser-Bessel** window. The Kaiser-Bessel window itself can be considered as an approximation of the so called **digital prolate spheroidal sequences** (DPSS, aka Slepian) window.

We will discuss these windows below in terms of their design criteria and the resulting additional parameter that can be set up to meet a desired constraint.
TBD...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack as fftpack
from scipy.signal import windows

## Dolph-Chebyshev Window

In [ ]:
# parameter for the window
M = 64  # window length
SidelobeAttenuation = 50  # in dB

In [ ]:
w = windows.chebwin(M, at=SidelobeAttenuation, sym=True)
# note on the code from:
# https://github.com/scipy/scipy/blob/v0.19.0/scipy/signal/windows.py#L1293-L1416
# 
# The tricky part here is that the analytic equations given in most textbooks
# such as e.g. R.G. Lyons (2011): "Understanding Digital Signal Processing",
# Prentice Hall, Upper Saddle River, 3rd ed., eq. (5.17)
# cannot be straightforwardly implemented due to numerical issues
# however, certain parts of the equation can be identified as Chebyshev polynomials
# (the window has its name from there) for which relations can be utilized.
# This is done in the scipy code for p[x < -1], p[np.abs(x) <= 1]

In [ ]:
plt.plot(w, 'o', color='C0')
plt.plot(w, color='C0')
plt.xlabel('sample k')
plt.ylabel('w[k]')
plt.title('Dolph-Chebyshev window')
plt.grid(True)

In [ ]:
Nz = 2**16  # zeropadding of window -> quasi-cont resolution of W for DTFT
Omega = 2*np.pi/Nz * np.arange(0,Nz)
wz = np.zeros(Nz)
wz[0:M] = w
Wz = fftpack.fft(wz)
Wzmax = np.max(20*np.log10(np.abs(Wz)))
plt.plot(Omega, 20*np.log10(np.abs(Wz))-Wzmax, 'k')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(fftpack.fftshift(Wz)))-Wzmax, 'C0')
plt.plot([np.pi, np.pi], [-120, 10], 'k')
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'$|W(\Omega)|$ in dB')
plt.xlim(-np.pi, 2*np.pi)
plt.ylim(-100, 10)
plt.title(['DTFT spectrum of Dolph-Chebyshev window with', M-1, 'zeros'])
plt.grid(True)

You might figure out the design criterion of the Dolph-Chebyshev by your own, when inspecting the DTFT spectrum of it. Hint: How is the additional parameter linked to the DTFT spectrum. Vary `M` and `SidelobeAttenuation` and check for changes. For small `M` (to make analysis convenient) check where the zeros are placed in the spectrum to meet the design criterion.

## Slepian Window

The design criterion of the **Slepian** (also named **digital prolate spheroidal sequences** (DPSS)) window is **maximum energy** concentration in the **main lobe** for a given mainlobe **bandwidth**. Actually, this is what you ask for, if no other specific constraints about the sidelobes positions and there levels are requested.

Recall, that the DTFT spectrum for the 'ideal world' window is the Dirac impulse (steming from the practically not feasible infinite rectangular window), so mainlobe energy concentration seems to be a very good approach to get close to it.

See

- Surendra Prasad (1982): "On an Index for Array Optimization and the Discrete Prolate Spheroidal Functions." In:
IEEE Transactions on Antennas and Propagation, vol. AP-30, no. 5, pg. 1021-1023, [DOI: 10.1109/TAP.1982.1142900](https://doi.org/10.1109/TAP.1982.1142900)

- Michael Möser (1988): "Analyse und Synthese akustischer Spektren.", Springer, Berlin, Kap. 3.2.2, [DOI: 10.1007/978-3-642-93374-5](https://doi.org/10.1007/978-3-642-93374-5)

- Julius O. Smith (2011): Spectral Audio Signal Processing, online lecture of CCRMA, Stanford University, https://ccrma.stanford.edu/~jos/sasp/Slepian_DPSS_Window.html


for treatments how to derive the Slepian window.

Challenging question: What window type results if you ask the Slepian window for mainlobe bandwidth $\rightarrow 0$? Implement a test case to approach the answer. 

In [ ]:
# parameter for the window
M = 64  # window length
bw = 2*np.pi/45  # -3dB bandwidth of the main lobe in terms of digital frequency
# empirically found for the specific window length

For the chosen example, this Slepian window has (i) the same window length and (ii) the first sidelobe is at about -50 dB like the Dolph-Chebyshev window above.

In [ ]:
w = windows.slepian(M, width=bw, sym=True)
plt.plot(w,'o',color='C0')
plt.plot(w,color='C0')
plt.xlabel('sample k')
plt.ylabel('w[k]')
plt.title('Slepian window')
plt.grid(True)

In [ ]:
Nz = 2**16  # zeropadding of window -> quasi-cont resolution of W for DTFT
Omega = 2*np.pi/Nz*np.arange(0,Nz)
wz = np.zeros(Nz)
wz[0:M] = w
Wz = fftpack.fft(wz)
Wzmax = np.max(20*np.log10(np.abs(Wz)))
plt.plot([np.pi, np.pi],[-120, 10], 'k')
plt.plot([-np.pi, 2*np.pi],[-3.01, -3.01], 'C1')
plt.plot([-np.pi, 2*np.pi],[-50, -50], 'dimgray')
plt.plot([-bw/2, -bw/2], [-120, 10], color='C1')
plt.plot([+bw/2, +bw/2], [-120, 10], color='C1')
plt.plot(Omega, 20*np.log10(np.abs(Wz))-Wzmax, 'k')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(fftpack.fftshift(Wz)))-Wzmax, 'C0')
plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'|W($\Omega$)| in dB')
plt.xlim(-np.pi, 2*np.pi)

# plt.xlim(-5*bw, +5*bw)  # zoom the main lobe

plt.ylim(-100, 10)
plt.title(['DTFT spectrum of Slepian window with', M-1, 'zeros'])
plt.grid(True)

## Kaiser-Bessel Window

The Kaiser-Bessel window is an **approximation** of the Slepian window **for large window lengths** $M$, note however that for they will be never identical. In the days of its **invention by Kaiser** it was much easier to compute it than the digital prolate spheroidal sequences. This is due to the explicit given equation for the Kaiser-Bessel window, whereas for the Slepian window an eigenwert problem for a $M/2$ matrix has to be numerically solved. The Kaiser-Bessel window requires the [zeroth-order modified **Bessel** function of the first kind](https://dlmf.nist.gov/10.25) $I_0(\cdot)$ to calculate the window signal $w[k]$. Thus, the given name in the DSP literature.

TBD...

## Comparison of the Windows

What are the differences between the discussed **parametric** window types of same length and same first sidelobe level?

In [ ]:
L = 1  # check for increasing integer L
# how Slepian and the Kaiser-Bessel approximation converge

M = 64*L
Nz = M*100  # zeropadding of window -> quasi-cont resolution of W for DTFT
k = np.arange(0,M)
bw = 2*np.pi/(45*L)
Omega = 2*np.pi/Nz * np.arange(0,Nz)
wdc = windows.chebwin(M, at=50, sym=True)  # dolph
wslep = windows.slepian(M, width=bw, sym=True)  # slepian
wkb = windows.kaiser(M, beta=6.85, sym=True)  # kaiser

Wdc = np.zeros(Nz)
Wdc[0:M] = wdc
Wdc = fftpack.fftshift(fftpack.fft(Wdc))
Wdc /= np.max(np.abs(Wdc))

Wslep = np.zeros(Nz)
Wslep[0:M] = wslep
Wslep = fftpack.fftshift(fftpack.fft(Wslep))
Wslep /= np.max(np.abs(Wslep))

Wkb = np.zeros(Nz)
Wkb[0:M] = wkb
Wkb = fftpack.fftshift(fftpack.fft(Wkb))
Wkb /= np.max(np.abs(Wkb))

plt.subplot(2,1,1)
plt.plot(k, wdc, label='Dolph')
plt.plot(k, wslep, label='Slepian')
plt.plot(k, wkb, label='Kaiser', color='C3')
plt.xlabel('sample k')
plt.ylabel('w[k]')
plt.title(['M=',M])
plt.legend()
plt.grid(True)

plt.subplot(2,1,2)
plt.plot((-np.pi,+np.pi), (-50,-50), color='dimgray')
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wdc)))
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wslep)))
plt.plot(Omega-np.pi, 20*np.log10(np.abs(Wkb)), color='C3')
plt.ylim(-100,0)
plt.xlim(-np.pi, +np.pi)

# plt.xlim(-3*bw,+3*bw)  # zoom the main lobe

plt.xlabel(r'$\Omega$ / rad')
plt.ylabel(r'W($\Omega$) in dB')
plt.grid(True)



**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises